In [0]:
import json
import sparknlp

sparknlp.start()

Out[5]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.166.247.57:7077 
 AppName 
 Databricks Shell

In [0]:
dffull = spark.read.format("avro").load("/mnt/scratch/BParticles17701850.avro")
dfads = spark.read.format("avro").load("/mnt/scratch/BPads17701850.avro")
dfERBM = spark.read.format("avro").load("/mnt/scratch/BPerbm4500.avro")
dftiny = spark.read.format("avro").load("/mnt/scratch/BPtinysample.avro")

In [0]:
# Selecting a database and selecting three relevant fields RecordID, text, and article title
# Change the line below to change the dataframe to work on
df = dffull
df = df.select ("RecordID" , "text", "ArticleTitle")

print(str(df.count()) + " articles available in the dataset")

981990 articles available in the dataset

In [0]:
# Filtering the dataframe based on the presence of certain words in the text
filteredDf = df.filter((df.text.rlike('(?i)libel')) & ( df.text.rlike('(?i)court') 
                      | df.text.rlike('(?i)case') | df.text.rlike('(?i)legal') 
                      | df.text.rlike('(?i)jury')))

numTexts = filteredDf.count()

print(str(numTexts) + " articles available in the filtered dataset")

13036 articles available in the filtered dataset

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline, PipelineModel


documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentenceDetector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

regexTokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token") \
    .setTargetPattern("\w+") \
    .setMinLength(3)  # splits at any non-alphanumeric character

stop_words = StopWordsCleaner.pretrained("stopwords_en", "en") \
    .setInputCols(["token"]) \
    .setOutputCol("cleanTokens")

normalizer = Normalizer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("normalized") \
    .setLowercase(True)

lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(["normalized"]) \
    .setOutputCol("lemma")

finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setIncludeMetadata(True)

pipeline = Pipeline().setStages([
    documentAssembler,
    sentenceDetector,
    regexTokenizer,
    stop_words,
    normalizer,
    lemmatizer,
    finisher 
])

stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[ | ][OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][OK!]

In [0]:
tokenized_df = pipeline.fit(filteredDf).transform(filteredDf)
tokenized_df.printSchema()

root
-- RecordID: string (nullable = true)
-- text: string (nullable = true)
-- ArticleTitle: string (nullable = true)
-- finished_lemma: array (nullable = true)
 |-- element: string (containsNull = true)
-- finished_lemma_metadata: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- _1: string (nullable = true)
 | |-- _2: string (nullable = true)

In [0]:
# Documentation - https://radimrehurek.com/gensim/auto_examples/index.html#core-tutorials-new-users-start-here
# Helpful sections - Corpora and Vector Spaces, Topics and Transformations

from gensim import corpora, models

# An array of arrays with each sub-array representing the list of lemma 
# found in a single document
texts = [row['finished_lemma'] for row in tokenized_df.take(3000)]

# Storing all words in a dictionary that will be used to assign the same unique
# token id to a word across all documents 
dct = corpora.Dictionary(texts)

# Stores the frequencies of all words occuring in a text as a sparse vector
# Ex. if a word 'libel' occurs 5 times in document 1 and is assigned an unique id of
# 23, corpus[0] will contain (23, 5). Use dct.token2id to see what id is assigned to a word
corpus = [dct.doc2bow(text) for text in texts]

tfidf = models.TfidfModel(corpus) # initialize a model

corpus_tfidf = tfidf[corpus]

lsi_model = models.LsiModel(corpus_tfidf, id2word= dct, num_topics=300)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

topics = lsi_model.print_topics(300)

In [0]:
# Iterating over list of correlation between each documents and all topics
# Each doc in this format [(0, 0.06600783396090518), (1, -0.520070330636184)]
documents = []
for doc in corpus_lsi:
  topicMatrix = []
  for topic in doc:
    topicMatrix.append((topics[topic[0]], topic[1] if topic[1] > 0 else -topic[1]))
    
  topicMatrix.sort(key = lambda x: x[1], reverse=True) # sort in descending order to get more related documents in front of list
  
  documents.append(topicMatrix)

In [0]:
from pprint import pprint

documentIdx = 0
numTopics = 5

pprint(documents[documentIdx][:numTopics]) # Prints 5 most related topics for that document

[((0,
 '0.113*"thc" + 0.083*"tbe" + 0.075*"house" + 0.072*"church" + 0.067*"bill" '
 '+ 0.060*"thle" + 0.060*"government" + 0.059*"tile" + 0.058*"lord" + '
 '0.055*"catholic"'),
 0.0791402522074277),
 ((9,
 '0.457*"tbe" + -0.204*"anel" + 0.156*"tbat" + 0.121*"defendant" + '
 '0.120*"tba" + -0.115*"sentence" + 0.110*"aad" + 0.105*"waa" + '
 '0.103*"catholic" + 0.093*"plaintiff"'),
 0.07449800313043035),
 ((261,
 '-0.044*"rubric" + 0.044*"wilkins" + -0.041*"locke" + 0.039*"kirk" + '
 '-0.038*"menemy" + 0.037*"jack" + -0.036*"cortes" + -0.036*"chairman" + '
 '-0.035*"beard" + -0.033*"sidney"'),
 0.06664723933089502),
 ((16,
 '0.298*"tbe" + -0.264*"thc" + -0.234*"plaintiff" + -0.212*"defendant" + '
 '-0.210*"xvas" + -0.154*"catholic" + 0.151*"anel" + 0.145*"petitioner" + '
 '0.111*"tbat" + 0.110*"slave"'),
 0.061764967461246024),
 ((213,
 '-0.058*"wakefield" + 0.051*"chap" + 0.050*"kean" + 0.047*"parr" + '
 '0.045*"dissenter" + -0.043*"phillips" + -0.043*"artery" + -0.042*"memoir" '
 '+ -0.040*"canada" + -0.038*"prynne"'),
 0.05796641714611582)]